In [1]:
import re
import time
import pickle
import urllib2
import numpy as np
from urllib2 import URLError
from bs4 import BeautifulSoup
import selenium.webdriver as webdriver
from selenium.common.exceptions import NoSuchElementException

In [2]:
# load and save lists and dictionaries with pickle
def save_obj(obj, name ):
    with open(''+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
def restaurant_insta_scraping_location(url):
    browser.get(url)
    try:
        browser.find_element_by_class_name('_oidfu').click()
        more_than_21 = 1
    except NoSuchElementException:
        more_than_21 = 0
    if more_than_21 == 1:
        for i in range(20):
            time.sleep(1.5)
            browser.execute_script("window.scrollTo(0,document.body.scrollHeight);") #Scroll down 
        time.sleep(1)
        html=browser.page_source
        try:
            pictable = browser.find_element_by_xpath('/html/body/span/section/main/article/div[2]/div[1]')
            is_there_pictable = 1
        except NoSuchElementException:
            is_there_pictable = 0
        if is_there_pictable == 1:
            rest_info = []
            for element in pictable.find_elements_by_css_selector("a[class='_8mlbc _vbtk2 _t5r8b']"):
                img = element.find_element_by_class_name('_icyx7')
                text = img.get_attribute('alt')
                tags = [t.replace("#","") for t in re.findall(r'#\w+', text)]
                #dic['url'] = img.get_attribute('src').encode('ascii')
                rest_info.append(tags)
        else:
            rest_info = 0
            
    else: 
        html=browser.page_source
        try:
            pictable = browser.find_element_by_xpath('/html/body/span/section/main/article/div[2]/div[1]')
            is_there_pictable = 1
        except:
            is_there_pictable = 0
        if is_there_pictable == 1:
            rest_info = []
            for element in pictable.find_elements_by_css_selector("a[class='_8mlbc _vbtk2 _t5r8b']"):
                img = element.find_element_by_class_name('_icyx7')
                text = img.get_attribute('alt')
                tags = [t.replace("#","") for t in re.findall(r'#\w+', text)]
                rest_info.append(tags)
        else:
            rest_info = 0
    return rest_info

def listHashtags(string): 
    hashtaglist = []
    while string.find('#',0) != -1:
        m = re.search('(?<=#)\w+', string)
        string = string.replace('#'+m.group(0),'')
        hashtaglist.append(m.group(0))
    hashtaglist = [item.lower() for item in hashtaglist]
    return hashtaglist

def image_insta_scraping(url):
    try:
        source = urllib2.urlopen(url)
        htmlcode = source.read()
        start = htmlcode.find('"caption":') + 12
        end = htmlcode.find('",',start)
        hash_list = listHashtags(htmlcode[start:end])
    except URLError:
        hash_list = 0
    return hash_list

def get_freq_sample(tags,dic):
    N = len(dic)
    freq_sample = [0]*N
    for tag in tags:
        try:
            freq_sample[dic.index(str(tag.lower()))] = 1
        except ValueError:
            None
    return freq_sample 

def two_point_distance(point1,point2):
    point1 = np.array(point1)
    point2 = np.array(point2)
    if len(point1.shape) != 1 or len(point2.shape) != 1 or point2.shape != point1.shape:
        print '(two_point_distance) ERROR: enter two valid points '
        return 'ERROR'
    resta = point1 - point2
    return np.sqrt(sum([el**2 for el in resta]))

In [4]:
def tag_recommender(url,clf,dic,known_rests_probs_mean):
    if 'locations' in url:
        tags_per_image = restaurant_insta_scraping_location(url)
        if tags_per_image == 0:
            return 'ERROR: non-valid URL'
        freq_samples_per_image = [get_freq_sample(tags,dic) for tags in tags_per_image]
        old_freq_samples_per_image = freq_samples_per_image
        freq_samples_per_image = [item for item in old_freq_samples_per_image if sum(item) != 0]
        if len(freq_samples_per_image) == 0:
            return 'ERROR: I cannot recommend this restaurant because I do not understand the tags of its images.'
        pred_probs_per_image = [clf.predict_proba([freq_samp])[0] for freq_samp in freq_samples_per_image]
        pred_probs_mean = (sum([np.array(pred_probs) for pred_probs in pred_probs_per_image])/len(pred_probs_per_image)).tolist()
        distances = [two_point_distance(rest[3],pred_probs_mean) for rest in known_rests_probs_mean]
        return known_rests_probs_mean[distances.index(min(distances))][2]
    else:
        if 'instagram.com' in url:
            image_tags = image_insta_scraping(url)
            if image_tags == 0:
                return 'ERROR: non-valid URL'
            freq_samp = get_freq_sample(image_tags,dic)
            if sum(freq_samp) == 0:
                return 'ERROR: I cannot recommend this image because I do not understand its tags.'
            else:
                pred_rest_type = clf.predict([freq_samp])
                pred_probs = clf.predict_proba([freq_samp])[0].tolist()
                distances = [two_point_distance(rest[3],pred_probs) for rest in known_rests_probs_mean]
                return known_rests_probs_mean[distances.index(min(distances))][2]
        else:
            print 'ERROR: non-valid URL'

In [5]:
max_n_of_images = 1000

In [6]:
rest_list = load_obj('rest_list')
clf = load_obj('clf_' + str(max_n_of_images))
dic = load_obj('dic_' + str(max_n_of_images))
known_rests_probs_mean = load_obj('known_rests_probs_mean_' + str(max_n_of_images))

In [154]:
# the recommender accepts either instagram image urls or instagram location urls:
#url = 'https://www.instagram.com/explore/locations/238741098/' # LA TAGLIATELLA (PASTA&PIZZA)
url = 'https://www.instagram.com/explore/locations/584346/' #YOSHINO (SUSHI)
#url = 'https://www.instagram.com/explore/locations/375802654/' #POCAS FOTOS
#url = 'https://www.instagram.com/p/BHqpEI7DfaG/' # KFC (FASTFOOD)
#url = 'https://www.instagram.com/explore/locations/269860565/'
#url = 'https://www.instagram.com/p/BHxFR58jPlh/?taken-at=269860565'
#url = 'https://www.instagram.com/explore/locations/215383179/'
#url = 'https://www.instagram.com/explore/locations/252007828/'
#url = 'https://www.instagram.com/explore/locations/234837112/'
#url = 'https://www.instagram.com/explore/locations/265379490/'

#path_to_chromedriver = '/Users/AlbertIribarne/Dropbox/UB/bigdata/instagram/chromedriver'
path_to_chromedriver = '/Users/martirossellogomez/Downloads/chromedriver'
browser = webdriver.Chrome(path_to_chromedriver)

recommendation = tag_recommender(url,clf,dic,known_rests_probs_mean)
print 'You should try: ' + recommendation

# print more info than the name
for rest in rest_list:
    if rest['NOMBRE'] == recommendation:
        print (rest['NOMBRE'],rest['COORDENADAS'],rest['ESPECIALIDAD'])

browser.quit()

You should try: Yoshino
('Yoshino', [41.385387, 2.151273], 'Cocina asiatica')


In [157]:
josep = []

In [158]:
for rest in rest_list:
    if rest['NOMBRE'] == recommendation:
        josep.append(rest['NOMBRE'])
        josep.append(rest['COORDENADAS'][0])
        josep.append(rest['COORDENADAS'][1])        
        josep.append(rest['ESPECIALIDAD'])
        josep.append(rest['CALLE'])
        try:
            for i in xrange(0,min(9,len(rest['IMAGES']))):
                josep.append(rest['IMAGES'][i]['url'])
        except KeyError:
            None

In [159]:
print josep

['Yoshino', 41.385387, 2.151273, 'Cocina asiatica', 'C de Mallorca, 90', 'https://instagram.fmad3-1.fna.fbcdn.net/t51.2885-15/s640x640/sh0.08/e35/13642857_567772963395396_946165459_n.jpg?ig_cache_key=MTI5MTMzOTE3NTY2NDUxODczOA%3D%3D.2', 'https://instagram.fmad3-1.fna.fbcdn.net/t51.2885-15/s640x640/sh0.08/e35/c135.0.810.810/13643600_1553001935007158_73898047_n.jpg?ig_cache_key=MTI5MDE0MjAyNjkxMzgxNDQ4Ng%3D%3D.2.c', 'https://instagram.fmad3-1.fna.fbcdn.net/t51.2885-15/s640x640/sh0.08/e35/13551545_530887097106554_1315725664_n.jpg?ig_cache_key=MTI4OTg3NDcwNTkzNTU0NzY4OA%3D%3D.2', 'https://instagram.fmad3-1.fna.fbcdn.net/t51.2885-15/s640x640/sh0.08/e35/c1.0.960.960/13556923_161645190920590_1879965918_n.jpg?ig_cache_key=MTI4OTM3NjkyMTk5NDE2MjM2Nw%3D%3D.2.c', 'https://instagram.fmad3-1.fna.fbcdn.net/t51.2885-15/e15/c157.0.405.405/13584143_1147240788652100_621154308_n.jpg?ig_cache_key=MTI4NzIwNzYzMjMyNzEyMjcwMA%3D%3D.2.c', 'https://instagram.fmad3-1.fna.fbcdn.net/t51.2885-15/s640x640/sh0.08/e3

In [160]:
print(len(josep))

14


In [161]:
import unidecode
from cartodb import CartoDBAPIKey, CartoDBException

josep[4].decode('unicode_escape').encode('ascii','ignore')

API_KEY ='2a85c1aab5f7c24a7306ee2acd6fadbcb0e97562'
cartodb_domain = 'instafoodub'
cl = CartoDBAPIKey(API_KEY, cartodb_domain)
if(len(josep)>=14):
    query = "UPDATE recommended SET nom = '%s', latitud = %s, longitud = %s, \
    tipus_cuina = '%s', carrer = '%s', image_1 = '%s', \
    image_2 = '%s', image_3 = '%s', image_4 = '%s', image_5 = '%s', \
    image_6 = '%s', image_7 = '%s', image_8 = '%s', image_9 = '%s' \
    " % (josep[0], josep[1], josep[2], josep[3], josep[0], \
    josep[5], josep[6], josep[7], josep[8], josep[9], \
    josep[10], josep[11], josep[12], josep[13])
else:
    while len(josep) < 14:
        josep.append('http://interbankcasapropia.pe/portalInmobiliario-theme/images/ibk/Imagen_no_disponible.jpg')
    query = "UPDATE recommended SET nom = '%s', latitud = %s, longitud = %s, \
    tipus_cuina = '%s', carrer = '%s', image_1 = '%s', \
    image_2 = '%s', image_3 = '%s', image_4 = '%s', image_5 = '%s', \
    image_6 = '%s', image_7 = '%s', image_8 = '%s', image_9 = '%s' \
    " % (josep[0].replace("'",''), josep[1], josep[2], josep[3], josep[4].replace("'",''), \
    josep[5], josep[6], josep[7], josep[8], josep[9], \
    josep[10], josep[11], josep[12], josep[13])
try:
    cl.sql(query)
except CartoDBException as e:
    print("some error ocurred", e)

In [162]:
try:
    cl.sql("UPDATE recommended SET the_geom = cdb_latlng (latitud , longitud) ")
except CartoDBException as e:
    print("some error ocurred", e)

In [165]:
from IPython.core.display import HTML
HTML('<iframe width="100%" height="520" frameborder="0" src="https://instafoodub.carto.com/viz/3d70bfac-4883-11e6-a172-0e233c30368f/embed_map" allowfullscreen webkitallowfullscreen mozallowfullscreen oallowfullscreen msallowfullscreen></iframe>')

In [153]:
save_obj(rest_list,'rest_list')